## Load OpenAI API Key

In [ ]:
import os
import nest_asyncio
import pandas as pd
nest_asyncio.apply()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)
assert OPENAI_API_KEY is not None, "Please set the OPENAI_API_KEY environment variable"

# Read the data that we created with the 1st notebook

In [ ]:
SAVE_PATH = "./papers_data"
corpus_dataset = f"{SAVE_PATH}/papers_corpus.parquet"
qa_train_dataset = f"{SAVE_PATH}/train_qa.parquet"
qa_test_dataset = f"{SAVE_PATH}/test_qa.parquet"
qa_df = pd.read_parquet(qa_train_dataset)

# TODO: Your choice. Must be smaller than NUM_QA used in the previous notebook
NUM_SAMPLES_TO_OPTIMIZE_RAG = -1

sample_qa_df = qa_df.sample(NUM_SAMPLES_TO_OPTIMIZE_RAG, random_state=42)
sample_qa_df = sample_qa_df.reset_index()
sample_qa_df.to_parquet(f"{SAVE_PATH}/sample_qa.parquet")

## Run evaluation

In [ ]:
from autorag.evaluator import Evaluator
PROJECT_DIR = "./project_dir" 

os.makedirs(PROJECT_DIR, exist_ok=True)

evaluator = Evaluator(
    qa_data_path=f"{SAVE_PATH}/sample_qa.parquet",
    corpus_data_path=f"{SAVE_PATH}/papers_corpus.parquet",
    project_dir=PROJECT_DIR
)

In [ ]:
# TODO: Complete TODOs in config.yaml
evaluator.start_trial('./config.yaml', skip_validation=True)

# Inspect results

In [ ]:
LAST_IDX = max([int(d) for d in os.listdir(PROJECT_DIR) if os.path.isdir(os.path.join(PROJECT_DIR, d)) and d.isdigit()])
print(f"Reading from {LAST_IDX}")
summary = pd.read_csv(f"{PROJECT_DIR}/{LAST_IDX}/summary.csv")
summary

In [ ]:
LAST_IDX = max([int(d) for d in os.listdir(PROJECT_DIR) if os.path.isdir(os.path.join(PROJECT_DIR, d)) and d.isdigit()])
print(f"Reading from {LAST_IDX}")

generator = pd.read_parquet(f"{PROJECT_DIR}/{LAST_IDX}/post_retrieve_node_line/generator/0.parquet")
best_generator = pd.read_parquet(f"{PROJECT_DIR}/{LAST_IDX}/post_retrieve_node_line/generator/best_0.parquet")
n = len(generator)
for i in range(n):
    print(f"Generator: {best_generator.iloc[i]['generated_texts']}")
    print("#"*50)


## Extract pipeline

- This will automatically detect the best combination of settings for your generator/retrieval/prompting components in your config.yaml to create best.yaml.

In [ ]:
from autorag.deploy import extract_best_config
LAST_IDX = max([int(d) for d in os.listdir(PROJECT_DIR) if os.path.isdir(os.path.join(PROJECT_DIR, d)) and d.isdigit()])
print(f"Reading from {LAST_IDX}")

extract_best_config(trial_path=f"{PROJECT_DIR}/{LAST_IDX}", output_path=f"{PROJECT_DIR}/{LAST_IDX}/best.yaml")

## Deploy your optimal RAG pipeline

In [ ]:
from autorag.deploy import Runner
LAST_IDX = max([int(d) for d in os.listdir(PROJECT_DIR) if os.path.isdir(os.path.join(PROJECT_DIR, d)) and d.isdigit()])
print(f"Reading from {LAST_IDX}")

runner = Runner.from_yaml(f"{PROJECT_DIR}/{LAST_IDX}/best.yaml", project_dir=PROJECT_DIR)

# TOOD: Ask your questions to the LLM+RAG model here!
runner.run("")